* `u.data`: Chứa toàn bộ các ratings của 943 _users_ cho 1682 movies. Mỗi user rate it nhất 20 movies. Thông tin về thời gian rate cũng được cho nhưng chúng ta không sử dụng trong bài viết này.

* `ua.base, ua.test, ub.base, ub.test`: là hai cách chia toàn bộ dữ liệu ra thành hai tập con, một cho training, một cho test. Chúng ta sẽ thực hành trên `ua.base` và `ua.test`. Bạn đọc có thể thử với cách chia dữ liệu còn lại.

* `u.user`: Chứa thông tin về _users_, bao gồm: id, tuổi, giới tính, nghề nghiệp, zipcode (vùng miền), vì những thông tin này cũng có thể ảnh hưởng tới sở thích của các _users_. Tuy nhiên, trong bài viết này, chúng ta sẽ không sử dụng các thông tin này, trừ thông tin về _id_ để xác định các user khác nhau.

* `u.genre`: Chứa tên của 19 thể loại phim. Các thể loại bao gồm: `unknown, Action, Adventure, Animation, Children's, Comedy, Crime, Documentary, Drama, Fantasy, Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western,`

* `u.item`: thông tin về mỗi bộ phim. Một vài dòng đầu tiên của file:
```
1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
```

In [ ]:
import pandas as pd
# Reading user file:
u_cols =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/content/u.user', sep='|', names=u_cols,
 encoding='latin-1')

n_users = users.shape[0]
print('Number of users:', n_users)
users.head() #uncomment this to see some few examples

Number of users: 943


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [ ]:
#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('/content/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('/content/ua.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.values
rate_test = ratings_test.values

print('Number of traing rates:', rate_train.shape[0])
print('Number of test rates:', rate_test.shape[0])

Number of traing rates: 90570
Number of test rates: 9430


In [ ]:
print(ratings_base.head())
print(ratings_test.head())

   user_id  movie_id  rating  unix_timestamp
0        1         1       5       874965758
1        1         2       3       876893171
2        1         3       4       878542960
3        1         4       3       876893119
4        1         5       3       889751712
   user_id  movie_id  rating  unix_timestamp
0        1        20       4       887431883
1        1        33       4       878542699
2        1        61       4       878542420
3        1       117       3       874965739
4        1       155       2       878542201


### Xây dựng item profiles

In [ ]:
#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('/content/u.item', sep='|', names=i_cols,
 encoding='latin-1')

n_items = items.shape[0]
print('Number of items:', n_items)
#print(items.head())

Number of items: 1682


In [ ]:
print(items.head())

   movie id        movie title release date  video release date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0       0   

   Adventure  Animation  Children's  ...  Fantasy  Film-Noir  Horror  Musical  \
0          0          1           1  ...        0          0       0        0

 19 giá trị nhị phân ở cuối mỗi hàng:

In [ ]:
X0 = items.values
X_train_counts = X0[:, -19:]
print(X0)
print(X_train_counts)

[[1 'Toy Story (1995)' '01-Jan-1995' ... 0 0 0]
 [2 'GoldenEye (1995)' '01-Jan-1995' ... 1 0 0]
 [3 'Four Rooms (1995)' '01-Jan-1995' ... 1 0 0]
 ...
 [1680 'Sliding Doors (1998)' '01-Jan-1998' ... 0 0 0]
 [1681 'You So Crazy (1994)' '01-Jan-1994' ... 0 0 0]
 [1682 'Scream of Stone (Schrei aus Stein) (1991)' '08-Mar-1996' ... 0 0
  0]]
[[0 0 0 ... 0 0 0]
 [0 1 1 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


xây dựng feature vector cho mỗi item dựa trên ma trận thể loại phim và TF-IDF (xem phần phụ lục).

In [ ]:
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()
print(tfidf)

[[0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.54 0.65 ... 0.54 0.   0.  ]
 [0.   0.   0.   ... 1.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]


mỗi hàng của `tfidf` tương ứng với feature vector của một bộ phim.

xây dựng những bộ phim nào mà _user_ đó đã _rated_, và giá trị của các _rating_ đó.

In [ ]:
import numpy as np
def get_items_rated_by_user(rate_matrix, user_id):
    """
    return (item_ids, scores)
    """
    y = rate_matrix[:,0] # all users
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1
    # but id in python starts from 0
    ids = np.where(y == user_id +1)[0]
    item_ids = rate_matrix[ids, 1] - 1 # index starts from 0
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)

Bây giờ, ta có thể đi tìm các hệ số của Ridge Regression cho mỗi _user_:

In [ ]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

d = tfidf.shape[1] # data dimension
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))

for n in range(n_users):
    ids, scores = get_items_rated_by_user(rate_train, n)
    clf = Ridge(alpha=0.01, fit_intercept  = True)
    Xhat = tfidf[ids, :]

    clf.fit(Xhat, scores)
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_


Sau khi tính được các hệ số `W` và `b`, _ratings_ cho mỗi _items_ được dự đoán bằng cách tính:

In [ ]:
# predicted scores
Yhat = tfidf.dot(W) + b

In [ ]:
print(Yhat)

[[2.92 4.   1.9  ... 5.04 4.78 3.17]
 [2.8  3.44 3.3  ... 1.1  4.76 3.7 ]
 [3.48 1.56 1.27 ... 7.36 4.19 3.81]
 ...
 [4.13 4.1  3.02 ... 9.54 4.64 3.27]
 [3.57 3.45 2.93 ... 3.68 3.66 2.76]
 [4.13 3.8  3.1  ... 9.54 4.42 3.93]]


In [ ]:
print(Yhat.shape)
print(Yhat[[221, 251, 280], 100])

(1682, 943)
[3.28 3.5  3.07]


Dưới đây là một ví dụ với _user_ có _id_ là `100`.

In [ ]:
n = 2
np.set_printoptions(precision=2) # 2 digits after .
ids, scores = get_items_rated_by_user(rate_test, n)
Yhat[n, ids]
print('Rated movies ids :', ids )
print('True ratings     :', scores)
print('Predicted ratings:', Yhat[ids, n])

Rated movies ids : [244 293 322 327 330 331 333 334 336 342]
True ratings     : [1 2 2 5 4 1 3 1 1 3]
Predicted ratings: [2.9  2.93 1.95 3.04 3.07 3.41 3.91 2.93 2.93 2.35]


Để đánh giá mô hình tìm được, chúng ta sẽ sử dụng Root Mean Squared Error (RMSE), tức căn bậc hai của trung bình cộng bình phương của lỗi. Lỗi được tính là hiệu của _true rating_ và _predicted rating_:

In [ ]:
from math import sqrt
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred
        se += (e*e).sum(axis = 0)
        cnt += e.size
    return sqrt(se/cnt)

print('RMSE for training:', evaluate(Yhat, rate_train, W, b))
print('RMSE for test    :', evaluate(Yhat, rate_test, W, b))

RMSE for training: 0.9089804562826721
RMSE for test    : 1.2703282700393035


Như vậy, với tập training, sai số vào khoảng 0.9 _sao_; với tập test, sai số lớn hơn một chút, rơi vào khoảng 1.3. Chúng ta thấy rằng kết quả này chưa thực sự tốt vì chúng ta đã đơn giản hoá mô hình đi quá nhiều. Kết quả tốt hơn có thể được thấy trong bài tiếp theo: Collaborative Filtering.

In [ ]:
# Danh sách các thể loại phim
genres = ["action", "drama", "comedy", "horror", "animation"]

# Thể loại của phim dưới dạng chuỗi
movie_genres_str = "action, drama"

# Chuyển đổi chuỗi thành danh sách các thể loại
movie_genres = movie_genres_str.split(", ")

# Tạo danh sách nhị phân dựa trên thể loại phim
binary_representation = [1 if genre in movie_genres else 0 for genre in genres]

# Hiển thị kết quả
print("Các thể loại phim:")
print(" ".join(genres))

print("\nDạng nhị phân:")
print(" ".join(map(str, binary_representation)))


Các thể loại phim:
action drama comedy horror animation

Dạng nhị phân:
1 1 0 0 0


In [ ]:
genres_binary = []

In [ ]:
genres_binary.append(binary_representation)
print(genres_binary[1])

[1, 1, 0, 0, 0]


In [ ]:
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(genres_binary).toarray()
print(tfidf)

[[0.38 0.38 0.   0.   0.85]
 [0.71 0.71 0.   0.   0.  ]
 [0.71 0.71 0.   0.   0.  ]
 [0.71 0.71 0.   0.   0.  ]
 [0.71 0.71 0.   0.   0.  ]
 [0.71 0.71 0.   0.   0.  ]]
